In [2]:
import pandas as pd
from collections import Counter
from sklearn.model_selection import train_test_split,KFold,cross_validate
import numpy as np
from sklearn import metrics
from sklearn.metrics import precision_score,roc_auc_score,accuracy_score
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import mlrose_hiive as mlrose
from sklearn.neural_network import MLPClassifier
import time

# #extract the data
raw_df = pd.read_csv('../Skin_NonSkin.txt', delimiter='	',names=["val1", "val2", "val3", "class"])
cleaned_df = raw_df.copy()


cleaned_df['class']=np.array([1 if x==1 else 0 for x in cleaned_df['class']])
print(cleaned_df.head(5))
#summarize data distribution
y=np.array(cleaned_df['class'])
X=np.array(cleaned_df.iloc[:,:-1])
print(Counter(y))

#rescale data
scaler = StandardScaler()
X = scaler.fit_transform(X)


   val1  val2  val3  class
0    74    85   123      1
1    73    84   122      1
2    72    83   121      1
3    70    81   119      1
4    70    81   119      1
Counter({0: 194198, 1: 50859})


In [ ]:

##cross validation to find out the best hidden_layer_size ###

t=5
cv = KFold(n_splits=t,shuffle=True)

RHC_AUC_test_score_bias_list=[]
RHC_AUC_test_score_variance_list=[]
SA_AUC_test_score_bias_list=[]
SA_AUC_test_score_variance_list=[]
GA_AUC_test_score_bias_list = []
GA_AUC_test_score_variance_list = []
RHC_AUC_train_score_bias_list=[]
SA_AUC_train_score_bias_list=[]
GA_AUC_train_score_bias_list=[]


hidden_layer_sizes = [(5,j,k) for j in range(6,8) for k in range(5,7)]

# parameters=(n_neighbors=5, *, weights='uniform', algorithm='auto', leaf_size=30, p=2, metric='minkowski', metric_params=None, n_jobs=None, **kwargs)[source]
for hidden_layer_size in hidden_layer_sizes:
    RHC_NN_model = mlrose.NeuralNetwork(hidden_nodes=hidden_layer_size, activation='tanh', algorithm='random_hill_climb', 
                                    max_iters=50, bias=True, is_classifier=True, 
                                    learning_rate=0.1, early_stopping=True, 
                                    clip_max=1e+10, restarts=10, max_attempts=10, random_state=3, curve=False)
    SA_NN_model = mlrose.NeuralNetwork(hidden_nodes=hidden_layer_size, activation='tanh', algorithm='simulated_annealing', 
                                    max_iters=50, bias=True, is_classifier=True, 
                                    learning_rate=0.1, early_stopping=True, 
                                    clip_max=1e+10, schedule= mlrose.GeomDecay(), max_attempts=10, random_state=3, curve=False)
    GA_NN_model = mlrose.NeuralNetwork(hidden_nodes=hidden_layer_size, activation='tanh', algorithm='genetic_alg', 
                                    max_iters=50, bias=True, is_classifier=True, 
                                    learning_rate=0.1, early_stopping=True, 
                                    clip_max=1e+10, pop_size=200, mutation_prob=0.1,max_attempts=10, random_state=3, curve=False)
#     MLP = MLPClassifier(hidden_layer_sizes=hidden_layer_size,activation ='tanh',solver='sgd',max_iter=200)
    RHC_AUC_test_score = []
    SA_AUC_test_score = []
    GA_AUC_test_score = []
    RHC_AUC_train_score = []
    SA_AUC_train_score = []
    GA_AUC_train_score = []
    

    for train_index, test_index in cv.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
#         start_time = time.time()
        RHC_NN_model.fit(X_train,y_train)
        print("RHC train finished")
#         RHC_training_time_total += time.time()-start_time()
#         start_time = time.time()
        SA_NN_model.fit(X_train,y_train)
        print("SA train finished")
#         SA_training_time_total += time.time()-start_time()
#         start_time = time.time()
        GA_NN_model.fit(X_train,y_train)
        print("GA train finished")
#         GA_training_time_total += time.time()-start_time()
        
        RHC_y_train_predict = RHC_NN_model.predict(X_train)
        RHC_y_train_predicted_probs = RHC_NN_model.predicted_probs
        SA_y_train_predict = RHC_NN_model.predict(X_train)
        SA_y_train_predicted_probs = RHC_NN_model.predicted_probs
        GA_y_train_predict = GA_NN_model.predict(X_train)
        GA_y_train_predicted_probs = GA_NN_model.predicted_probs
        RHC_AUC_train_score.append(roc_auc_score(y_train,RHC_y_train_predicted_probs))
        SA_AUC_train_score.append(roc_auc_score(y_train,SA_y_train_predicted_probs))
        GA_AUC_train_score.append(roc_auc_score(y_train,GA_y_train_predicted_probs))
        
        RHC_y_test_predict = RHC_NN_model.predict(X_test)
        RHC_y_test_predicted_probs = RHC_NN_model.predicted_probs
        SA_y_test_predict = SA_NN_model.predict(X_test)
        SA_y_test_predicted_probs = SA_NN_model.predicted_probs
        GA_y_test_predict = GA_NN_model.predict(X_test)
        GA_y_test_predicted_probs = GA_NN_model.predicted_probs
        RHC_AUC_test_score.append(roc_auc_score(y_test,RHC_y_test_predicted_probs))
        SA_AUC_test_score.append(roc_auc_score(y_test,SA_y_test_predicted_probs))
        GA_AUC_test_score.append(roc_auc_score(y_test,GA_y_test_predicted_probs)) 
        
   
    RHC_AUC_train_score_bias_list.append(np.array(RHC_AUC_train_score).mean())
    RHC_AUC_test_score_bias_list.append(np.array(RHC_AUC_test_score).mean())
    RHC_AUC_test_score_variance_list.append(np.array(RHC_AUC_test_score).std()**2)
    
    SA_AUC_train_score_bias_list.append(np.array(SA_AUC_train_score).mean())
    SA_AUC_test_score_bias_list.append(np.array(SA_AUC_test_score).mean())
    SA_AUC_test_score_variance_list.append(np.array(SA_AUC_test_score).std()**2)
    
    GA_AUC_train_score_bias_list.append(np.array(GA_AUC_train_score).mean())
    GA_AUC_test_score_bias_list.append(np.array(GA_AUC_test_score).mean())
    GA_AUC_test_score_variance_list.append(np.array(GA_AUC_test_score).std()**2)
    

#generate tunning graph#######
# m = len(hidden_layer_sizes)
hidden_layer_sizes_text=[str(hidden_layer_size[0])+str(hidden_layer_size[1])+str(hidden_layer_size[2]) for hidden_layer_size in hidden_layer_sizes]
fig_0, axs = plt.subplots(1, 2, figsize=(10, 5), sharey=False)
axs[0].plot(hidden_layer_sizes_text,RHC_AUC_test_score_bias_list,"r^",linestyle = "--",label='RHC AUC_Bias')
axs[0].plot(hidden_layer_sizes_text,SA_AUC_test_score_bias_list,"b^",linestyle = "--",label='SA AUC_Bias')
axs[0].plot(hidden_layer_sizes_text,GA_AUC_test_score_bias_list,"g^",linestyle = "--",label='GA AUC_Bias')

axs[1].plot(hidden_layer_sizes_text,RHC_AUC_test_score_variance_list,"r^",linestyle = "--",label='RHC AUC_Variance')
axs[1].plot(hidden_layer_sizes_text,SA_AUC_test_score_variance_list,"b^",linestyle = "--",label='SA AUC_Variance')
axs[1].plot(hidden_layer_sizes_text,GA_AUC_test_score_variance_list,"g^",linestyle = "--",label='GA AUC_Variance')

axs[0].set_xlabel("hidden_layer_size")
axs[0].set_ylabel("AUC_Bias")
axs[1].set_xlabel("hidden_layer_size")
axs[1].set_ylabel("AUC_Variance")

axs[0].legend()
axs[1].legend()

fig_0.suptitle("Three random NN model: Bias/Variance Comparison(NN tunning max_iter=50)")
fig_0.savefig("Three random NN model: Bias-Variance Comparison.png")



fig_1,axs = plt.subplots(1,3,figsize=(15,5),sharey=False)
axs[0].plot(hidden_layer_sizes_text,RHC_AUC_train_score_bias_list,"r^",linestyle = "--",label='training data')
axs[0].plot(hidden_layer_sizes_text,RHC_AUC_test_score_bias_list,"b^",linestyle = "--",label='test data')


axs[1].plot(hidden_layer_sizes_text,SA_AUC_train_score_bias_list,"r^",linestyle = "--",label='training data')
axs[1].plot(hidden_layer_sizes_text,SA_AUC_test_score_bias_list,"b^",linestyle = "--",label='test data')

axs[2].plot(hidden_layer_sizes_text,GA_AUC_train_score_bias_list,"r^",linestyle = "--",label='training data')
axs[2].plot(hidden_layer_sizes_text,GA_AUC_test_score_bias_list,"b^",linestyle = "--",label='test data')
axs[0].set_xlabel("hidden_layer_size")
axs[0].set_ylabel("RHC_AUC_score")
axs[1].set_xlabel("hidden_layer_size")
axs[1].set_ylabel("SA_AUC_score")
axs[2].set_xlabel("hidden_layer_size")
axs[2].set_ylabel("GA_AUC_score")
axs[0].legend()
axs[1].legend()
axs[2].legend()
fig_1.suptitle("Three NN models' AUC_score(NN tuning max_iter=50)")
fig_1.savefig("Three NN models' AUC_score(NN tuning max_iter=50).png")




In [ ]:
###compare the learning curve with MLP with different training size#########

MLP_Accuracy_train_score=[]
MLP_Accuracy_test_score=[]
RHC_Accuracy_train_score=[]
RHC_Accuracy_test_score =[]
SA_Accuracy_train_score=[]
SA_Accuracy_test_score=[]
GA_Accuracy_train_score=[]
GA_Accuracy_test_score=[]
MLP_Precision_test_score=[]
MLP_Precision_train_score=[]
RHC_Precision_test_score=[]
RHC_Precision_train_score=[]
SA_Precision_test_score=[]
SA_Precision_train_score=[]
GA_Precision_test_score=[]
GA_Precision_train_score=[]
MLP_training_time = []
RHC_training_time = []
SA_training_time = []
GA_training_time = []
MLP_training_time =[]
for k in range(4,11):
    MLP = MLPClassifier(hidden_layer_sizes=(5,7,6),activation ='tanh',solver='sgd',max_iter=50)
    
    RHC_NN_model = mlrose.NeuralNetwork(hidden_nodes=(5,7,5), activation='tanh', algorithm='random_hill_climb',
                                    max_iters=50, bias=True, is_classifier=True,
                                    learning_rate=0.1, early_stopping=True,
                                    clip_max=1e+10, restarts=10, max_attempts=10, random_state=3, curve=False)
    SA_NN_model = mlrose.NeuralNetwork(hidden_nodes=(5,7,5), activation='tanh', algorithm='simulated_annealing',
                                    max_iters=50, bias=True, is_classifier=True,
                                    learning_rate=0.1, early_stopping=True,
                                    clip_max=1e+10, schedule= mlrose.GeomDecay(), max_attempts=10, random_state=3, curve=False)
    GA_NN_model = mlrose.NeuralNetwork(hidden_nodes=(5,7,6), activation='tanh', algorithm='genetic_alg',
                                    max_iters=50, bias=True, is_classifier=True,
                                    learning_rate=0.1, early_stopping=True,
                                    clip_max=1e+10, pop_size=200, mutation_prob=0.1,max_attempts=10, random_state=3, curve=False)
    print(k)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 1/k, random_state = 3)
    
    start_time=time.time()
    MLP.fit(X_train,y_train)
    MLP_training_time.append(time.time()-start_time)
    print("MLP train finished")
    MLP_y_train_predict = MLP.predict(X_train)
    MLP_y_test_predict = MLP.predict(X_test)
    MLP_Accuracy_train_score.append(accuracy_score(y_train,MLP_y_train_predict))
    MLP_Accuracy_test_score.append(accuracy_score(y_test,MLP_y_test_predict))
    MLP_Precision_test_score.append(precision_score(y_test,MLP_y_test_predict))
    MLP_Precision_train_score.append(precision_score(y_train,MLP_y_train_predict))
    
    start_time = time.time()
    RHC_NN_model.fit(X_train,y_train)
    RHC_training_time.append(time.time()-start_time)
    print("RHC train finished")
    RHC_y_train_predict = RHC_NN_model.predict(X_train)
    RHC_y_test_predict = RHC_NN_model.predict(X_test)
    RHC_Accuracy_train_score.append(accuracy_score(y_train,RHC_y_train_predict))
    RHC_Accuracy_test_score.append(accuracy_score(y_test,RHC_y_test_predict))
    RHC_Precision_test_score.append(precision_score(y_test,RHC_y_test_predict))
    RHC_Precision_train_score.append(precision_score(y_train,RHC_y_train_predict))
   
   
    start_time = time.time()
    SA_NN_model.fit(X_train,y_train)
    SA_training_time.append(time.time()-start_time)
    print("SA train finished")
    SA_y_train_predict = SA_NN_model.predict(X_train)
    SA_y_test_predict = SA_NN_model.predict(X_test)
    SA_Accuracy_train_score.append(accuracy_score(y_train,SA_y_train_predict))
    SA_Accuracy_test_score.append(accuracy_score(y_test,SA_y_test_predict))
    SA_Precision_test_score.append(precision_score(y_test,SA_y_test_predict))
    SA_Precision_train_score.append(precision_score(y_train,SA_y_train_predict))
   
   
   
    start_time = time.time()
    GA_NN_model.fit(X_train,y_train)
    GA_training_time.append(time.time()-start_time)
    print("GA train finished")
    GA_y_train_predict = GA_NN_model.predict(X_train)
    GA_y_test_predict = GA_NN_model.predict(X_test)
    GA_Accuracy_train_score.append(accuracy_score(y_train,GA_y_train_predict))
    GA_Accuracy_test_score.append(accuracy_score(y_test,GA_y_test_predict))
    GA_Precision_test_score.append(precision_score(y_test,GA_y_test_predict))
    GA_Precision_train_score.append(precision_score(y_train,GA_y_train_predict))
   
   

##genearte learning curves with traning size#######

K= range(4,11)
fig_0, axs = plt.subplots(1, 1, figsize=(5, 5), sharey=False)
axs.plot(K,RHC_training_time,"bo",linestyle = "--",label="RHC")
axs.plot(K,SA_training_time,"y^",linestyle = "--",label="SA")
axs.plot(K,GA_training_time,"go",linestyle = "--",label="GA")
axs.plot(K,MLP_training_time,'r^',linestyle = "--",label="MLP")
axs.set_xlabel("training size k")
axs.set_ylabel("training time")
axs.legend()
fig_0.suptitle("Randomized Weight Optimization vs MLP: Training time")
fig_0.savefig("Randomized Weight Optimization vs MLP: Training time.png")

fig_1, axs = plt.subplots(1, 4, figsize=(10, 10), sharey=False)

axs[0].plot(K,RHC_Accuracy_train_score,"r^",linestyle = "--",label='training data')
axs[0].plot(K,RHC_Accuracy_test_score,"b^",linestyle = "--",label='test data')
axs[1].plot(K,SA_Accuracy_train_score,"r^",linestyle = "--",label='training data')
axs[1].plot(K,SA_Accuracy_test_score,"b^",linestyle = "--",label='test data')
axs[2].plot(K,GA_Accuracy_train_score,"r^",linestyle = "--",label='training data')
axs[2].plot(K,GA_Accuracy_test_score,"b^",linestyle = "--",label='test data')
axs[3].plot(K,MLP_Accuracy_train_score,"r^",linestyle = "--",label='training data')
axs[3].plot(K,MLP_Accuracy_test_score,"b^",linestyle = "--",label='test data')

axs[0].set_xlabel("training size k")
axs[0].set_ylabel("RHC accuracy_score")
axs[1].set_xlabel("training size k")
axs[1].set_ylabel("SA accuracy_score")
axs[2].set_xlabel("training size k")
axs[2].set_ylabel("GA accuracy_score")
axs[3].set_xlabel("training size k")
axs[3].set_ylabel("MLP accuracy_score")
axs[0].legend()
axs[1].legend()
axs[2].legend()
axs[3].legend()

fig_1.suptitle("Randomized Optimization vs MLP: Accuracy score")
fig_1.savefig("Randomized Optimization vs MLP: Accuracy score.png")

fig_2, axs = plt.subplots(1, 4, figsize=(10, 10), sharey=False)

axs[0].plot(K,RHC_Precision_train_score,"r^",linestyle = "--",label='training data')
axs[0].plot(K,RHC_Precision_test_score,"b^",linestyle = "--",label='test data')
axs[1].plot(K,SA_Precision_train_score,"r^",linestyle = "--",label='training data')
axs[1].plot(K,SA_Precision_test_score,"b^",linestyle = "--",label='test data')
axs[2].plot(K,GA_Precision_train_score,"r^",linestyle = "--",label='training data')
axs[2].plot(K,GA_Precision_test_score,"b^",linestyle = "--",label='test data')
axs[3].plot(K,MLP_Precision_train_score,"r^",linestyle = "--",label='training data')
axs[3].plot(K,MLP_Precision_test_score,"b^",linestyle = "--",label='test data')

axs[0].set_xlabel("training size k")
axs[0].set_ylabel("RHC precision_score")
axs[1].set_xlabel("training size k")
axs[1].set_ylabel("SA precision_score")
axs[2].set_xlabel("training size k")
axs[2].set_ylabel("GA precision_score")
axs[3].set_xlabel("training size k")
axs[3].set_ylabel("MLP precision_score")
axs[0].legend()
axs[1].legend()
axs[2].legend()
axs[3].legend()
fig_2.suptitle("Randomized Optimization vs MLP : Precision score")
fig_2.savefig("Randomized Optimization vs MLP: Precision score.png")

###each model's bias/variance
RHC_Accuracy_mean = np.array(RHC_Accuracy_test_score).mean()
RHC_Accuracy_variance = np.array(RHC_Accuracy_test_score).std()**2
SA_Accuracy_mean = np.array(SA_Accuracy_test_score).mean()
SA_Accuracy_variance = np.array(SA_Accuracy_test_score).std()**2
GA_Accuracy_mean = np.array(GA_Accuracy_test_score).mean()
GA_Accuracy_variance = np.array(GA_Accuracy_test_score).std()**2
MLP_Accuracy_mean=np.array(MLP_Accuracy_test_score).mean()
MLP_Accuracy_variance = np.array(MLP_Accuracy_test_score).std()**2

RHC_Precision_mean = np.array(RHC_Precision_test_score).mean()
RHC_Precision_variance = np.array(RHC_Precision_test_score).std()**2
SA_Precision_mean = np.array(SA_Precision_test_score).mean()
SA_Precision_variance = np.array(SA_Precision_test_score).std()**2
GA_Precision_mean = np.array(GA_Precision_test_score).mean()
GA_Precision_variance = np.array(GA_Precision_test_score).std()**2
MLP_Precision_mean=np.array(MLP_Precision_test_score).mean()
MLP_Precision_variance = np.array(MLP_Precision_test_score).std()**2

fig_3, axs = plt.subplots(1, 4, figsize=(10, 10), sharey=False)
X=["RHC","SA","GA","MLP"]
axs[0].bar(X,[RHC_Accuracy_mean,SA_Accuracy_mean,GA_Accuracy_mean,MLP_Accuracy_mean])
axs[1].bar(X,[RHC_Accuracy_variance,SA_Accuracy_variance,GA_Accuracy_variance,MLP_Accuracy_variance])
axs[2].bar(X,[RHC_Precision_mean,SA_Precision_mean,GA_Precision_mean,MLP_Precision_mean])
axs[3].bar(X,[RHC_Precision_variance,SA_Precision_variance,GA_Precision_variance,MLP_Precision_variance])
axs[0].set_ylabel("Accuracy Bias")
axs[1].set_ylabel("Accuracy Variance")
axs[2].set_ylabel("Precision Bias")
axs[3].set_ylabel("Precision Variance")
fig_3.suptitle("Accuracy and Precision Bias/Variance")
fig_3.savefig("Accuracy and Precision Bias-Variance.png")

